In [1]:
import plotly.express as px
import plotly.graph_objects as go
from api.api import API
api = API('')
df=api.reports('SELECT * FROM PIX_MOVEMENTS',False)

In [10]:
from plotly.subplots import make_subplots

In [6]:
total=df.groupby('status')['pix_amount'].count()
status=df.groupby(['status','in_or_out'])['id'].count()

In [40]:
# fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "bar"}]],
#                     subplot_titles=("Total Failed vs Total Completed", "Total Failed per type of transaction"))
# fig.add_trace(
#     go.Pie(labels=total.index, values=total.values,pull=[0, 0.2], domain=dict(x=[0, 0.5])),row=1, col=1
# )

# fig.add_trace(
#     go.Bar(x=status['failed'].index,y=status['failed'].values, showlegend=False),row=1, col=2
# )
# fig.update_layout(title_text='PIX Transaction Status Metrics 2020', title_x=0.5,
#                   legend=dict(x=0,y=0.7))
# [fig.layout.annotations[i].update(y=1.05) for i in range(2)]
# fig


In [42]:
df

,id,account_id,in_or_out,pix_amount,pix_requested_at,pix_completed_at,status
0,13629077-0946-8179-9680-000000000000,50928183-6645-3152-6400-000000000000,pix_out,1894.77,1579693633580,1.579694e+12,completed
1,22467941-1802-2659-0720-000000000000,50928183-6645-3152-6400-000000000000,pix_out,419.79,1587309244550,1.587309e+12,completed
2,16569602-6045-6373-7600-000000000000,50928183-6645-3152-6400-000000000000,pix_in,943.61,1609306218320,1.609306e+12,completed
3,91954593-2589-0464-0000-000000000000,50928183-6645-3152-6400-000000000000,pix_in,124.66,1586015734430,1.586016e+12,completed
4,20012121-4224-0523-0080-000000000000,50928183-6645-3152-6400-000000000000,pix_out,775.96,1590974667000,1.590975e+12,completed
...,...,...,...,...,...,...,...
249877,22684146-9023-7561-3440-000000000000,67496194-7612-9720-3200-000000000000,pix_out,1596.65,1579357052400,1.579357e+12,completed
249878,31521285-1824-8888-3200-000000000000,67496194-7612-9720-3200-000000000000,pix_out,453.79,1581142919280,1.581143e+12,completed
249879,18005771-4051-6179-9680-000000000000,67496194-7612-9720-3200-000000000000,pix_out,45.89,1608954790060,1.608955e+12,completed
249880,16081458-3687-7373-1200-000000000000,67496194-7612-9720-3200-000000000000,pix_in,1165.90,1598609653080,1.598610e+12,completed


In [79]:
status_hour="""
WITH TIME AS (
SELECT TIME_ID,
(CASE WHEN action_timestamp::timestamp::time<='15:00:00'::time AND action_timestamp::timestamp::time>='09:00:00'::time THEN 'business_hour'
ELSE 'non_business_hour' END) IS_BUSINESS_HOUR
from d_time
)
SELECT ACCOUNT_ID,IN_OR_OUT,PIX_AMOUNT,PIX.STATUS,TIME.IS_BUSINESS_HOUR
FROM PIX_MOVEMENTS PIX
JOIN TIME
ON PIX.PIX_REQUESTED_AT=TIME.TIME_ID 
WHERE STATUS='failed'
"""
status_hour_t="""
WITH TIME AS (
SELECT TIME_ID,
(CASE WHEN action_timestamp::timestamp::time<='15:00:00'::time AND action_timestamp::timestamp::time>='09:00:00'::time THEN 'business_hour'
ELSE 'non_business_hour' END) IS_BUSINESS_HOUR
from d_time
),
T_IN AS (
	SELECT IS_BUSINESS_HOUR,ACCOUNT_ID,AMOUNT,STATUS
	FROM TRANSFER_INS TI
	JOIN TIME
	ON TI.TRANSACTION_REQUESTED_AT=TIME.TIME_ID 
	WHERE TI.STATUS='failed'
	),
	T_OUT AS (
	SELECT IS_BUSINESS_HOUR,ACCOUNT_ID,AMOUNT,STATUS
	FROM TRANSFER_OUTS TOT
	JOIN TIME
	ON TOT.TRANSACTION_REQUESTED_AT=TIME.TIME_ID 
	WHERE TOT.STATUS='failed'
	)

SELECT *
FROM T_IN
UNION
SELECT * 
FROM T_OUT

"""

In [80]:
dfh=api.reports(status_hour,False)
dft=api.reports(status_hour_t,False)


In [82]:
dfh['is_business_hour'].value_counts()

non_business_hour    9361
business_hour        3238
Name: is_business_hour, dtype: int64

In [81]:
dft['is_business_hour'].value_counts()

non_business_hour    9323
business_hour        3038
Name: is_business_hour, dtype: int64

In [83]:
a=dfh['is_business_hour'].value_counts()
b=dft['is_business_hour'].value_counts()

In [89]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
colors=['rgb(18, 36, 37)','rgb(56, 75, 126)' ]
fig.add_trace(go.Pie(labels=a.index, values=a.values,pull=[0, 0.2], marker_colors=colors),
              1, 1)
fig.add_trace(go.Pie(labels=b.index, values=b.values,pull=[0, 0.2]),
              1, 2)
